In [1]:
import requests
import pandas as pd
from datetime import datetime

now_date = datetime.now().strftime("%d.%m.%Y")

# Get data
alerts_url = f'https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx?lang=he&fromDate=01.01.2014&toDate={now_date}&mode=0'
alerts_json = requests.get(alerts_url).json()

# Break multi-region alerts into separate records
df = pd.DataFrame.from_records(alerts_json)
df['data'] = df['data'].str.split(',')
df = df.explode('data')

# Remove sub-regions such as א, ב, ג, ד
df = df[df['data'].str.len() > 2]

# Change Hatzor to detailed name as the google geocoder fail to detect the correct city
df['data'] = df['data'].replace('חצור', 'חצור אשדוד')

# Show rocket sirens only
df = df[df['category_desc'] == 'ירי רקטות וטילים']

# Map city names to coordinates
def get_coordinates(city_name):
    city_name = city_name + ', ישראל'
    api_key = 'YOUR_API_KEY'
    geocoder_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={city_name}&key={api_key}&language=iw'
    geocoding_result = requests.get(geocoder_url).json()
    lat = geocoding_result['results'][0]['geometry']['location']['lat']
    long = geocoding_result['results'][0]['geometry']['location']['lng']
    return(lat, long)
    
city_to_coords = {}
for city in df['data'].unique():
    try:
        city_to_coords[city] = get_coordinates(city)
        print(city, '\t -', city_to_coords[city], end='\r')
    except Exception as e:
        city_to_coords[city] = (None, None)
        print(city, '\t', '- failed finding coordinates')

# Apply mapping on all data
df['outLat'] = df['data'].apply(lambda x: city_to_coords[x][0])
df['outLong'] = df['data'].apply(lambda x: city_to_coords[x][1])

# Fixed Gaza coordinates
df['inLat'] = 31.513
df['inLong'] = 34.452

# Filter wrong coordinates outside of Israel polygon (only if you use a bad geocoder)
'''
filtered_df = df[(df['outLong'] < 35.8)
                & (df['outLong'] > 33.3)
                & (df['outLat'] < 34.0)
                & (df['outLat'] > 29.2)]

filtered_df.to_csv('RocketLaunchData - Filtered.csv', encoding='utf-8-sig', index=False)
display(filtered_df)
'''

df.to_csv('RocketLaunchData.csv', encoding='utf-8-sig', index=False)
display(df)

 שובה 	 - failed finding coordinates6)675)34.554535)


,data,date,time,datetime,category,category_desc,matrix_id,rid,outLat,outLong,inLat,inLong
0,נתיב העשרה,07.08.2022,19:17:22,2022-08-07T19:17:00,1,ירי רקטות וטילים,1,15119,31.572407,34.539322,31.513,34.452
1,יד מרדכי,07.08.2022,19:17:22,2022-08-07T19:17:00,1,ירי רקטות וטילים,1,15118,31.586963,34.557500,31.513,34.452
2,כרם שלום,07.08.2022,19:15:32,2022-08-07T19:16:00,1,ירי רקטות וטילים,1,15117,31.228281,34.284559,31.513,34.452
3,כרם שלום,07.08.2022,19:07:03,2022-08-07T19:07:00,1,ירי רקטות וטילים,1,15116,31.228281,34.284559,31.513,34.452
4,מטווח ניר עם,07.08.2022,19:05:51,2022-08-07T19:06:00,1,ירי רקטות וטילים,1,15115,31.518459,34.580544,31.513,34.452
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,באר שבע - מערב,17.05.2021,17:46:47,2021-05-17T17:47:00,1,ירי רקטות וטילים,1,13001,31.252973,34.791462,31.513,34.452
1996,באר שבע - דרום,17.05.2021,17:46:47,2021-05-17T17:47:00,1,ירי רקטות וטילים,1,13000,31.252973,34.791462,31.513,34.452
1997,שתולים,17.05.2021,17:46:42,2021-05-17T17:47:00,1,ירי רקטות וטילים,1,12999,31.773687,34.684409,31.513,34.452
1998,שדה עוזיהו,17.05.2021,17:46:42,2021-05-17T17:47:00,1,ירי רקטות וטילים,1,12998,31.758437,34.678194,31.513,34.452


In [3]:
# Install instructions available at - https://docs.kepler.gl/docs/keplergl-jupyter#install
from keplergl import KeplerGl

config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['OrefSirens'],
     'id': 'ed77mhj1',
     'name': ['datetime'],
     'type': 'timeRange',
     'value': [1659732960000, 1659739300000],
     'enlarged': True,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1}],
   'layers': [{'id': 'pd16gsv',
     'type': 'arc',
     'config': {'dataId': 'OrefSirens',
      'label': 'SirensEdges',
      'color': [255, 251, 152],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'inLat',
       'lng0': 'inLong',
       'lat1': 'outLat',
       'lng1': 'outLong'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [218, 0, 0]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': 'p5giket',
     'type': 'heatmap',
     'config': {'dataId': 'OrefSirens',
      'label': 'SirensNodes',
      'color': [231, 159, 213],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'outLat', 'lng': 'outLong'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'weightField': None, 'weightScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'OrefSirens': [{'name': 'data',
        'format': None},
       {'name': 'date', 'format': None},
       {'name': 'time', 'format': None},
       {'name': 'datetime', 'format': None},
       {'name': 'category', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': -12.14678899082569,
   'dragRotate': True,
   'latitude': 31.36529666166856,
   'longitude': 34.579642984864655,
   'pitch': 52.32949284907476,
   'zoom': 9.2,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': True,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

kepler_map = KeplerGl(config=config)
kepler_map.add_data(data=df, name='OrefSirens')

with open('kepler.html', 'wb') as file:
    file.write(kepler_map._repr_html_())

kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['OrefSirens'], 'id': 'ed77mh…